In [ ]:
df = pd.read_csv('data/training_set/log_mini.csv', dtype={'skip_1':int, 'skip_2':int, 'skip_3':int, 'not_skipped':int, 'hist_user_behavior_is_shuffle':int, 'premium':int})
tf = pd.read_csv('data/track_features/tf_mini.csv')

NameError: name 'pd' is not defined

In [2]:
import pandas as pd
import random
random.seed(5483)
import numpy as np
np.random.seed(5483)
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'pandas'

In [3]:
cols_to_select = [
    'context_switch',
    'context_type',
    'hist_user_behavior_is_shuffle',
    'hist_user_behavior_n_seekback',
    'hist_user_behavior_n_seekfwd',
    'hist_user_behavior_reason_end',
    'hist_user_behavior_reason_start',
    'hour_of_day',
    'long_pause_before_play',
    'no_pause_before_play',
    'not_skipped',
    'premium',
    'session_position',
    'short_pause_before_play',
    'skip_1',
    'skip_2',
    'skip_3',
    'dayofweek']
cols_tf = ['duration', 'release_year', 'us_popularity_estimate',
       'acousticness', 'beat_strength', 'bounciness', 'danceability',
       'dyn_range_mean', 'energy', 'flatness', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mechanism', 'organism', 'speechiness',
       'tempo', 'time_signature', 'valence', 'acoustic_vector_0',
       'acoustic_vector_1', 'acoustic_vector_2', 'acoustic_vector_3',
       'acoustic_vector_4', 'acoustic_vector_5', 'acoustic_vector_6',
       'acoustic_vector_7']
cols_log = ['session_id', 'session_position', 'session_length', 'track_id_clean',
       'skip_1', 'skip_2', 'skip_3', 'not_skipped', 'context_switch',
       'no_pause_before_play', 'short_pause_before_play',
       'long_pause_before_play', 'hist_user_behavior_n_seekfwd',
       'hist_user_behavior_n_seekback', 'hist_user_behavior_is_shuffle',
       'hour_of_day', 'date', 'premium', 'context_type',
       'hist_user_behavior_reason_start', 'hist_user_behavior_reason_end',
       'dayofweek']
data_df = df.loc[lambda x: x.session_length==20]
log_features = data_df.loc[lambda x: x.session_position <= (x.session_length/2)].copy()
log_features.date = pd.to_datetime(log_features.date)
log_features['dayofweek'] = log_features.date.dt.dayofweek
replace_idx = log_features.hist_user_behavior_reason_start.isin(['endplay', 'popup', 'uriopen', 'clickside']).copy()
log_features.loc[replace_idx, 'hist_user_behavior_reason_start'] = 'replaced'
replace_idx = log_features.hist_user_behavior_reason_end.isin(['clickrow', 'appload', 'popup', 'uriopen', 'clickside', 'logout']).copy()
log_features.loc[replace_idx, 'hist_user_behavior_reason_end'] = 'replaced'
tf.rename(columns={'track_id': 'track_id_clean'}, inplace=True)
features = pd.merge(log_features, tf, on=['track_id_clean'], how='inner')
features.sort_values(['session_id', 'session_position'], inplace=True)
features.reset_index(drop=True, inplace=True)
ground_truth = data_df.loc[lambda x: x.session_position > (x.session_length/2)].loc[:, ['session_id', 'session_position', 'session_length', 'track_id_clean', 'skip_2']].copy()
ground_truth.sort_values(['session_id', 'session_position'], inplace=True)
ground_truth.reset_index(drop=True, inplace=True)
pred_y = np.array(ground_truth.groupby('session_id')['skip_2'].apply(lambda x: x.tolist()).tolist())

feats_dummies = pd.get_dummies(features.loc[:, cols_to_select])
feats_track = features.loc[:, cols_tf]
all_seq = np.array(features.reset_index().groupby('session_id')['index'].apply(lambda x: x.tolist()).tolist())
y_seq = np.array(features.reset_index().groupby('session_id')['skip_2'].apply(lambda x: x.tolist()).tolist())[:, -1:]
all_seq = np.concatenate((all_seq, y_seq), axis=1)
cols_order = sorted(feats_dummies.columns.values)
log_mat = feats_dummies.loc[:, cols_order].values
track_mat = feats_track.values
X_train, X_test, y_train, y_test = train_test_split(all_seq, pred_y, test_size=0.2, random_state=5483)



NameError: name 'df' is not defined

In [4]:
from keras.layers import Embedding, BatchNormalization, Input, Dense, LSTM, concatenate, Bidirectional, RepeatVector, Dropout, Lambda
import keras.backend as K
from keras.optimizers import adam_v2
from keras.models import Model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

ModuleNotFoundError: No module named 'keras'

In [5]:
def model_net(log_mat, track_mat):
  seq_input = Input(shape=(None,), name='log_input')
  log_embed = Embedding(input_dim=log_mat.shape[0], output_dim=log_mat.shape[1], weights=[log_mat], trainable=False, mask_zero=False, name='session_embed')
  track_embed = Embedding(input_dim=track_mat.shape[0], output_dim=track_mat.shape[1], weights=[track_mat],trainable=False,mask_zero=False,name='track_embed')
  log_bn = BatchNormalization(name='bn1')
  log_transformer = Dense(64, activation='relu', name='session_transformer')
  x1 = log_embed(seq_input)
  x1 = log_bn(x1)
  x1 = log_transformer(x1)
  track_bn = BatchNormalization(name='track_bn')
  track_transformer = Dense(64, activation='relu', name='track_transformer')
  x2 = track_embed(seq_input)
  x2 = track_bn(x2)
  x2 = track_transformer(x2)
  x = concatenate([x1, x2], axis=-1)
  x = Bidirectional(LSTM(64, return_sequences=False, return_state=False), name='lstm1')(x)
  x = RepeatVector(1)(x)
  x_en = Dense(256, activation='relu', name='base_transformer')(x)
  _, fwd_sh, fwd_sc, bck_sh, bck_sc = Bidirectional(LSTM(256, return_sequences=False, return_state=True))(x_en)
  fwd_sh = Dropout(0.2)(fwd_sh)
  fwd_sc = Dropout(0.2)(fwd_sc)
  bck_sh = Dropout(0.2)(bck_sh)
  bck_sc = Dropout(0.2)(bck_sc)
  y_p = Input(shape=(1, 1), name='y_seq')
  all_ouputs = []
  decoder_2 = Bidirectional(LSTM(256, return_sequences=True, return_state=True, name='decoder_2'))
  decoder_3 = LSTM(64, return_sequences=True, return_state=True, name='decoder_3')
  decoder_4 = Dropout(0.3)
  decoder_5 = Dense(1, activation='sigmoid', name='decoder_5')
  x = concatenate([x_en, y_p])
  x, fwd_sh, fwd_sc, bck_sh, bck_sc = decoder_2(x, initial_state=[fwd_sh, fwd_sc, bck_sh, bck_sc])
  x, sh, sc = decoder_3(x)
  x = decoder_4(x)
  out = decoder_5(x)
  all_ouputs.append(out)

  for i in range(1, 10):
    x = concatenate([x_en, out])
    x, fwd_sh, fwd_sc, bck_sh, bck_sc = decoder_2(x, initial_state=[fwd_sh, fwd_sc, bck_sh, bck_sc])
    x, sh, sc = decoder_3(x)
    x = decoder_4(x)
    out = decoder_5(x)
    all_ouputs.append(out)
  out_combined = Lambda(lambda x: K.concatenate(x, axis=1))(all_ouputs)
  model = Model(inputs=[seq_input, y_p], outputs=[out_combined])
  model.compile(optimizer=adam_v2.Adam(lr=0.01), loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
model = model_net(log_mat, track_mat)
model.summary()

NameError: name 'log_mat' is not defined

In [7]:
callbacks = [
    EarlyStopping(monitor='val_loss',
                     patience=15,
                     verbose=1,
                     min_delta=1e-4,
                     mode='min'),
    ReduceLROnPlateau(monitor='val_loss',
                         factor=0.1,
                         patience=4,
                         verbose=1,
                         epsilon=1e-4,
                         mode='min'),
    ModelCheckpoint(monitor='val_loss',
                       filepath='model_weights_best.h5',
                       save_best_only=True,
                       save_weights_only=True)
]

NameError: name 'EarlyStopping' is not defined

In [8]:
history=model.fit(
    x=[X_train[:, :-1], X_train[:, -1:]], 
    y=y_train, 
    batch_size=128, 
    epochs=100, 
    verbose=1, 
    callbacks=callbacks, 
    validation_split=0.1, 
    shuffle=True)


NameError: name 'model' is not defined

In [ ]:
print(history.history['acc'])
print(history.history['loss'])
print(history.history)
epochs = range(1, len(acc) + 1)


In [9]:
model.load_weights('model_weights_best.h5')
loss, acc = model.evaluate(
    x=[X_test[:, :-1], X_test[:, -1:]], 
    y=y_test
)
print('Test Loss:{}.\nTest Accuracy:{}.'.format(loss, acc))

NameError: name 'model' is not defined